In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost
import pandas as pd
from tqdm import tqdm
from datetime import datetime as dpa
import pickle

from src.utils.convert_data_type import convertStrtoDate, convertToDayWeek, convertToHour
from src.utils.encodeData import labelEncoder

data = pd.read_excel('/home/ai_car/AthenaBow WanD/data/FinalData_Simulate.xlsx')
data = data.drop(columns=['STT', 'CallID', 'Hotline', 'QC User Say', 'QC Intent', 'Đánh giá Intent', 'Tỷ lệ nhận diện STT', 'Đánh giá STT', 'Audio', 'Đơn giá', 'CUSTOMER'])
data = data.drop(columns=['Intent', 'Độ tự tin (%)', 'Entities', 'Entity Value', 'QC Độ trễ', 'Số khách hàng', 'Mã kết thúc bot'])
data = data.drop(columns=['Action', 'Bot Process (ms)', 'TTS Process (ms)', 'User Say', 'Time in call', 'ASR Process (ms)'])
data = data.drop(columns=['Unnamed: 0']) 
# data = data.drop(columns=['CURRES_PHONE'])

In [2]:
data = data.drop(columns=['TENURE', 'RELATIONSHIP_2', 'PHONE_REF3', 'NAME_3', 'RELATIONSHIP_3', 'PERMNENT_PROVINCE'])
data = data.drop(columns=['MOB', 'RELATIONSHIP_1', 'PHONE_REF2', 'NAME_2', 'DPD', 'REMAINING_PEROID'])
data = data.drop(columns=['NAME_1', 'DISBURSALDATE', 'PHONE_REF1', 'MAX_DPD', 'DUEDAY', 'FIRST_PAID_DATE'])
data = data.drop(columns=['Mã tổng đài', 'AMOUNT_FINANCE', 'GENDER', 'Thời lượng gọi'])
data = data.drop(columns=['AGREEMENTID'])
data = data.drop_duplicates(ignore_index= True)
data

,Thời điểm gọi,Trạng thái,CURRES_PHONE,PRODUCT,NC,Channel
0,2022-08-20 08:32:26,Nghe máy,399739449.0,SERVICE PACKAGE LOAN,9824414,4
1,2022-08-20 08:32:26,Nghe máy,399739449.0,SERVICE PACKAGE LOAN,9824414,1
2,2022-08-20 08:32:26,Nghe máy,399739449.0,SERVICE PACKAGE LOAN,9824414,3
3,2022-08-20 08:32:26,Nghe máy,399739449.0,SERVICE PACKAGE LOAN,9824414,2
4,2022-08-20 08:32:26,Nghe máy,382868777.0,SERVICE PACKAGE LOAN,8803001,3
...,...,...,...,...,...,...
77665,2022-08-30 20:59:10,Không thành công,367006699.0,CONSUMER DURABLE LOAN,1500130,4
77666,2022-08-30 20:59:10,Không thành công,333149288.0,Cash Loan,1499884,1
77667,2022-08-30 20:59:15,Không thành công,944121431.0,CONSUMER DURABLE LOAN,1499135,2
77668,2022-08-30 20:59:25,Không thành công,901327305.0,Cash Loan,1497872,1


In [3]:
data = data.dropna()
data = data.reset_index(drop=True)
def convertPhone(data):
    listPhone = []
    for i in tqdm(range(len(data['CURRES_PHONE']))):
        listPhone.append(int(data['CURRES_PHONE'][i]))
    return listPhone
data['CURRES_PHONE'] = convertPhone(data)

100%|██████████| 77668/77668 [00:00<00:00, 177822.52it/s]


In [4]:
data.rename(columns = {'Trạng thái':'Result'}, inplace = True)
def encodeTimeCall(data):
    listEncode = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        a = dpa.strptime(data['Thời điểm gọi'][i], '%Y-%m-%d %H:%M:%S')
        listEncode.append(a)
    return listEncode
data['Thời điểm gọi'] = encodeTimeCall(data)

100%|██████████| 77668/77668 [00:01<00:00, 71548.94it/s]


In [5]:
def convertDateYeartoDays(data):
    DayofWeeks = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        DayofWeeks.append(convertToDayWeek(data, i))
    return DayofWeeks
data['DayofWeek'] = convertDateYeartoDays(data)

100%|██████████| 77668/77668 [00:02<00:00, 37284.42it/s]


In [6]:
def convertToTime(data):
    hourToCall = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        hourToCall.append(convertToHour(data, i))
    return hourToCall
data['TimeCall'] = convertToTime(data)

100%|██████████| 77668/77668 [00:00<00:00, 112835.97it/s]


In [7]:
def labelEncoder(data):
    encoderLabel = []
    for i in tqdm(range(len(data['Result']))):
        if data['Result'][i] == 'Không thành công' or data['Result'][i] == 'Lỗi':
            encoderLabel.append(0)
        elif data['Result'][i] == 'Nghe máy':
            encoderLabel.append(1)
    return encoderLabel

data['Result'] = labelEncoder(data)

100%|██████████| 77668/77668 [00:00<00:00, 93696.30it/s] 


In [8]:
def encodeProduct(data):
    encoder = []
    for i in tqdm(range(len(data['PRODUCT']))):
        if data['PRODUCT'][i] == 'Cash Loan':
            encoder.append(0)
        elif data['PRODUCT'][i] == 'TOPUP - XSELL':
            encoder.append(1)
        elif data['PRODUCT'][i] == 'CONSUMER DURABLE LOAN':
            encoder.append(2)
        elif data['PRODUCT'][i] == 'TWO-WHEEL LOAN':
            encoder.append(3)
        elif data['PRODUCT'][i] == 'SERVICE PACKAGE LOAN':
            encoder.append(5)
    return encoder
data['PRODUCT'] = encodeProduct(data)

100%|██████████| 77668/77668 [00:00<00:00, 127238.83it/s]


In [9]:
def convertToSecond(data):
    hourToCall = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        hourToCall.append(convertToS(data, i))
    return hourToCall
data['TimeCall'] = convertToTime(data)

100%|██████████| 77668/77668 [00:00<00:00, 112818.39it/s]


In [14]:
print(f"{data.TimeCall.isna().sum()}, {data.PRODUCT.isna().sum()}, {data.NC.isna().sum()}")
print(data.Result.value_counts())
data = data.drop(columns=['Thời điểm gọi'])
data

0, 0, 0
1    48458
0    29210
Name: Result, dtype: int64


KeyError: "['Thời điểm gọi'] not found in axis"

In [16]:
data = data.drop(columns=['DayofWeek'])

In [17]:
y = data.Result
x = data.drop(columns=['Result'])
x, y

(       CURRES_PHONE  PRODUCT       NC  Channel  TimeCall
 0         399739449        5  9824414        4         8
 1         399739449        5  9824414        1         8
 2         399739449        5  9824414        3         8
 3         399739449        5  9824414        2         8
 4         382868777        5  8803001        3         8
 ...             ...      ...      ...      ...       ...
 77663     367006699        2  1500130        4        20
 77664     333149288        0  1499884        1        20
 77665     944121431        2  1499135        2        20
 77666     901327305        0  1497872        1        20
 77667     822800708        0  1495538        4        20
 
 [77668 rows x 5 columns],
 0        1
 1        1
 2        1
 3        1
 4        1
         ..
 77663    0
 77664    0
 77665    0
 77666    0
 77667    0
 Name: Result, Length: 77668, dtype: int64)

In [18]:
CallCenter = xgboost.DMatrix(data=x, label=y)

In [19]:
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
from xgboost.sklearn import XGBClassifier
import numpy as np
import xgboost as xgb
from matplotlib.pylab import rcParams
from sklearn import metrics
rcParams['figure.figsize'] = 12, 4

In [20]:
target = 'Result'
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(data= dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors].values, dtrain['Result'].values,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Result'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Result'], dtrain_predprob))
                    

In [21]:
predictors = [a for a in data.columns if a not in [target]]
xgb1 = XGBClassifier(tree_method='gpu_hist', gpu_id=0,
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, data, predictors)

/home/ai_car/anaconda3/envs/Python3.7/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Model Report
Accuracy : 0.7618
AUC Score (Train): 0.828380


In [22]:
pickle.dump(xgb1, open("/home/ai_car/AthenaBow/model/xgb_without_weekday.sav", "wb"))
xgb_reg = pickle.load(open("/home/ai_car/AthenaBow/model/xgb_without_weekday.sav", "rb"))


In [ ]:
399739449        5  9824414        4          7         8

In [27]:
preds = xgb_reg.predict([[399739449,  5, 9824414,4  , 8]])
preds

array([0])

In [20]:
predictors

['CURRES_PHONE', 'PRODUCT', 'NC', 'Channel', 'DayofWeek', 'TimeCall']

In [15]:
for k in range(0,5):
    print(k)

0
1
2
3
4


In [60]:
s = [2, 3, 1, 4, 5]
li=[]
 
for i in range(len(s)):
      li.append([s[i],i,i])
li.sort(reverse=True)
sort_index = []
 
for x in li:
      sort_index.append(x[2])
 
print(sort_index)

[4, 3, 1, 0, 2]


In [61]:
from datetime import datetime
import time
import pickle

start = time.time()

xgb_reg = pickle.load(open("/home/ai_car/AthenaBow/model/xgb_without_weekday.sav", "rb"))
data_exam = [[312800708, 1, 5985332],[3523308, 0, 2185112],[3124121431, 2, 7365334],[55559739449, 1, 6315334],[3231208, 5, 1983134]]
now = datetime.now()
week_day = now.weekday() + 2
list_result = []

for j in range(len(data_exam)):
    one_result = []
    for i in range(7,23):
        for k in range(1,5):
            #preds = xgb_reg.predict([data_exam[j] + [week_day, i]])
            pred = xgb_reg.predict_proba([data_exam[j] + [k, i]]).tolist()
            
            max_index = pred[0].index(max(pred[0]))
            if(max_index == 1):
                one_result.append([max(pred[0]),i,k])
    one_result.sort(reverse=True)
    #print(one_result)   
    #sort_result = sorted(one_result, reverse=True)
    list_result.append([one_result[0],one_result[1],one_result[2]])
    
print(list_result)

end = time.time() - start
print(end)

[[[0.9816898107528687, 15, 1], [0.9532426595687866, 15, 3], [0.9325528740882874, 15, 2]], [[0.9518262147903442, 15, 1], [0.9229453802108765, 15, 3], [0.9097697138786316, 15, 2]], [[0.9927117824554443, 15, 3], [0.9918099045753479, 15, 1], [0.9898322820663452, 15, 2]], [[0.9630601406097412, 15, 1], [0.9554781317710876, 15, 3], [0.9227817058563232, 15, 2]], [[0.8138037323951721, 10, 1], [0.7881088256835938, 10, 2], [0.7100624442100525, 8, 4]]]
0.5451700687408447


In [51]:
list_result[1]

[[0.9514403343200684, 15, 1],
 [0.9016355276107788, 15, 2],
 [0.8039489984512329, 15, 3]]

In [54]:
from datetime import datetime
import time
import pickle

start = time.time()

xgb_reg = pickle.load(open("/home/ai_car/AthenaBow/model/noMinutes/xgb_simulated.sav", "rb"))
data_exam = [[3216305, 7, 5985332],[3523308, 9, 5985112],[3216308, 2, 5365334],[3216417, 1, 6315334],[3231208, 9, 5983134]]
now = datetime.now()
week_day = now.weekday() + 2
list_result = []
list_result_index = []
list_result_channel = []
for j in range(0,len(data_exam)):
    one_result = []
    one_result_index = []
    one_result_channel = []
    for i in range(6,24):
        for k in range(1,5):
            #preds = xgb_reg.predict([data_exam[j] + [week_day, i]])
            pred = xgb_reg.predict_proba([data_exam[j] + [k,7, i]]).tolist()
            
            max_index = pred[0].index(max(pred[0]))
            if(max_index == 1):
                one_result.append(max(pred[0]))
                one_result_index.append(i)
                one_result_channel.append(k)
    sort_result = sorted(one_result, reverse=True)
    
    list_result.append(sort_result[0])
    list_result_index.append(one_result_index[one_result.index(sort_result[0])])
    list_result_channel.append(one_result_channel[one_result.index(sort_result[0])])
    list_result.append(sort_result[1])
    list_result_index.append(one_result_index[one_result.index(sort_result[1])])
    list_result_channel.append(one_result_channel[one_result.index(sort_result[1])])
    list_result.append(sort_result[2])
    list_result_index.append(one_result_index[one_result.index(sort_result[2])])
    list_result_channel.append(one_result_channel[one_result.index(sort_result[2])])
    print(list_result)
    print(list_result_index)
    print(list_result_channel)
# print(list_result)
# print(list_result_index)
# print(list_result_channel)
end = time.time() - start
print(end)

[0.8867882490158081, 0.7611852288246155, 0.738341748714447, 0.6151707768440247, 0.8867882490158081, 0.7611852288246155, 0.738341748714447, 0.6151707768440247, 0.8867882490158081, 0.7611852288246155, 0.738341748714447, 0.6151707768440247, 0.7411207556724548, 0.5237669944763184, 0.5001811385154724, 0.7183668613433838, 0.5667483806610107, 0.5082535147666931, 0.737568736076355, 0.6534894704818726, 0.6043796539306641, 0.737568736076355, 0.6534894704818726, 0.6043796539306641, 0.7821367979049683, 0.6484958529472351, 0.6630798578262329, 0.5345214605331421, 0.7666347622871399, 0.6241685152053833, 0.6419339776039124, 0.5111505389213562, 0.9087724089622498, 0.8196401596069336, 0.8233900666236877, 0.7075477838516235]
[0.9087724089622498, 0.8867882490158081, 0.8867882490158081, 0.8867882490158081, 0.8233900666236877, 0.8196401596069336, 0.7821367979049683, 0.7666347622871399, 0.7611852288246155, 0.7611852288246155, 0.7611852288246155, 0.7411207556724548, 0.738341748714447, 0.738341748714447, 0.738

In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, date
import pickle
from time import time
from src.utils import libSort
import calendar
from datetime import datetime as dpa
from calendar import monthrange
from src.utils.convert_data_type import convertList, convertToHour
from src.utils.QueryData import GetPhone, checkNumberDayRemaining, SearchInfor, GetMinute, GetIndex
from src.utils.mergeToDataFrame import format_time

database = pd.read_csv('/home/ai_car/AthenaBow/data/TotalData.csv')
xgbModel = pickle.load(open('/home/ai_car/AthenaBow/model/24-12/xgb.sav', 'rb'))
# database = database.drop(columns=['STT', 'CallID', 'Hotline', 'QC User Say', 'QC Intent', 'Đánh giá Intent', 'Tỷ lệ nhận diện STT', 'Đánh giá STT', 'Audio', 'Đơn giá', 'CUSTOMER'])
# database = database.drop(columns=['Intent', 'Độ tự tin (%)', 'Entities', 'Entity Value', 'QC Độ trễ', 'Số khách hàng', 'Mã kết thúc bot'])
# database = database.drop(columns=['Action', 'Bot Process (ms)', 'TTS Process (ms)', 'User Say', 'Time in call', 'ASR Process (ms)'])
# database = database.drop(columns=['TENURE', 'RELATIONSHIP_2', 'PHONE_REF3', 'NAME_3', 'RELATIONSHIP_3', 'PERMNENT_PROVINCE', 'RELATIONSHIP_1', 'PHONE_REF2', 'NAME_2'])
# database = database.drop(columns=['MOB', 'DPD', 'REMAINING_PEROID'])
# database = database.drop(columns=['NAME_1', 'DISBURSALDATE', 'PHONE_REF1', 'MAX_DPD', 'DUEDAY', 'FIRST_PAID_DATE'])
# database = database.drop(columns=['Mã tổng đài', 'AMOUNT_FINANCE', 'GENDER', 'Thời lượng gọi'])
# database = database.drop_duplicates(ignore_index= True)
# database = database.dropna(axis=1)
# database = database.reset_index(drop=True)
def convertPhone(database):
    listPhone = []
    for i in tqdm(range(len(database['CURRES_PHONE']))):
        if str(database['CURRES_PHONE'][i]) == "nan":
            listPhone.append(0)
        else:
            listPhone.append(int(database['CURRES_PHONE'][i]))
    return listPhone
database['CURRES_PHONE'] = convertPhone(database)
def encodeTimeCall(data):
    listEncode = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        a = dpa.strptime(data['Thời điểm gọi'][i], '%Y-%m-%d %H:%M:%S')
        listEncode.append(a)
    return listEncode
database['Thời điểm gọi'] = encodeTimeCall(database)
def convertToTime(data):
    hourToCall = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        hourToCall.append(convertToHour(data, i))
    return hourToCall
database['TimeCall'] = convertToTime(database)
listTimeEncode = []
for i in tqdm(range(len(database['TimeCall']))):
    listTimeEncode.append(int(int(database['TimeCall'][i]) * 60) + GetMinute(database, i))
database['TimeCall'] = listTimeEncode
database.rename(columns = {'Trạng thái':'Result'}, inplace = True)
def labelEncoder(data):
    encoderLabel = []
    for i in tqdm(range(len(data['Result']))):
        if data['Result'][i] == 'Không thành công' or data['Result'][i] == 'Lỗi':
            encoderLabel.append(0)
        elif data['Result'][i] == 'Nghe máy':
            encoderLabel.append(1)
    return encoderLabel

database['Result'] = labelEncoder(database)


/home/ai_car/anaconda3/envs/Python3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (8,9,13,14,15,17,18,25,36,37,40,43,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 1699122/1699122 [00:17<00:00, 94398.75it/s] 


In [10]:
listResult

,26/12/2022 - Monday,27/12/2022 - Tuesday,28/12/2022 - Wednesday,29/12/2022 - Thursday,30/12/2022 - Friday
0,"(87.223, 690)","(88.622, 690)","(89.196, 690)","(88.99, 690)","(89.601, 675)"
1,"(86.718, 675)","(88.183, 675)","(88.956, 675)","(88.746, 675)","(89.502, 690)"
2,"(86.398, 660)","(88.124, 630)","(88.499, 660)","(88.282, 660)","(89.315, 660)"
3,"(86.056, 630)","(87.893, 660)","(88.088, 630)","(88.28, 630)","(89.063, 630)"
4,"(85.821, 600)","(87.775, 645)","(88.068, 540)","(88.185, 645)","(88.889, 645)"
5,"(85.513, 540)","(87.594, 600)","(88.068, 525)","(88.087, 600)","(88.559, 615)"
6,"(85.513, 525)","(87.467, 540)","(87.896, 585)","(87.846, 540)","(88.518, 585)"
7,"(85.449, 585)","(87.467, 525)","(87.767, 555)","(87.846, 525)","(88.349, 600)"
8,"(85.385, 555)","(87.354, 555)","(87.483, 600)","(87.54, 555)","(88.124, 540)"
9,"(84.267, 615)","(87.261, 585)","(86.983, 645)","(87.441, 585)","(88.124, 525)"


In [2]:
# listAgreedmentID = 2662516
#2523722
                    # 2662516,
                    # 1661663,
                    # 2599746,
                    # 2421024]

listAgreedmentID = 1524786    
# for i in range(len(listAgreedmentID)):
listID = GetIndex(data= database, id= listAgreedmentID)
Phone, Product, Nc = SearchInfor(data= database, index= listID[0], NCIndex= listID[-1])
historyCall = GetPhone(data=database, Phone= Phone)
NowDay, LimitInMonth, NowMonth = checkNumberDayRemaining()
whatInsideToModel = [[Phone, Product, Nc]]
listResult = pd.DataFrame()
for i in tqdm(range(len(whatInsideToModel))):
    for j in range(26,31):
        perCent = []
        listTimeAvailabel = []
        for k in tqdm(range(480, 12001, 15)):
            preds = xgbModel.predict_proba([[whatInsideToModel[i][0],
                                            whatInsideToModel[i][1],
                                            whatInsideToModel[i][2],
                                            k,
                                            j,
                                            NowMonth]])
            preds = preds.tolist()
            perCent.append(preds[0][1])
            listTimeAvailabel.append(k)
        whenIsGoodTime = libSort.nlargest(10,zip(perCent, listTimeAvailabel))
        whenIsGoodTime = convertList(whenIsGoodTime)
        StandardScaleTime = []
        for ff in range(len(whenIsGoodTime)):
            HereWeLive = (f"{round(float(whenIsGoodTime[ff][0] * 100),3)}", whenIsGoodTime[ff][1])
            StandardScaleTime.append(HereWeLive)
        DayinWeeks = date(year= datetime.date(datetime.now()).year,
                            month=NowMonth,
                            day= int(j))
        listResult[f'{j}/{NowMonth}/{datetime.date(datetime.now()).year} - {calendar.day_name[DayinWeeks.weekday()]}'] = StandardScaleTime


100%|██████████| 1/1 [00:06<00:00,  6.96s/it]


In [8]:
maxPercent = float(85)
Statictis = []
for m in listResult.columns:
    flag = True
    flag2 = False
    countTotals = 0
    countReach = 0
    topFit = []
    for n in range(len(listResult[m])):
        countTotalsInTime = 0
        countReachInTime = 0
        if float(listResult[m][n][0]) < maxPercent:
            if flag2 == False:
                flag = False
                break
            else:
                flag == True
        else:
            flag = True
            extractDay = int(m.split('/')[0])
            for p in range(len(historyCall)):
                dayExecuse = int(database['Thời điểm gọi'][historyCall[p]].day)
                if dayExecuse == extractDay:
                    countTotals = countTotals + 1
                    if int(listResult[m][n][1]) in range(int(database['TimeCall'][historyCall[p]]) - 60,int(database['TimeCall'][historyCall[p]]) + 60):
                        countTotalsInTime = countTotalsInTime + 1
                        if int(database['Result'][historyCall[p]]) == 1:
                            countReachInTime = countReachInTime + 1
                            countReach = countReach + 1
                            flag2 = True
                    else:
                        if int(database['Result'][historyCall[p]]) == 1:
                            countReach = countReach + 1
                            flag2 = True
        if flag2 == True:
            if countTotalsInTime == 0:
                BestofDay = {'Time to Call' : format_time(listResult[m][n][1]),
                                'Total Call' : countTotals,
                                'Total Reach In Day' : countReach,
                                'Total Call In Time' : countTotalsInTime,
                                'Total Reach In Time' : countReachInTime,
                                'ConfidentInDays' : (countReach/countTotals) * 100}
                                # 'ConfidentInTimes' : 0}
            else:
                BestofDay = {'Time to Call' : format_time(listResult[m][n][1]),
                                'Total Call' : countTotals,
                                'Total Reach In Day' : countReach,
                                'Total Call In Time' : countTotalsInTime,
                                'Total Reach In Time' : countReachInTime,
                                'ConfidentInDays' : (countReach/countTotals) * 100,
                                'ConfidentInTimes' : countReachInTime/countTotalsInTime *100}
            topFit.append(BestofDay)
    if flag == True:
        ResultIHope = {str(m) : topFit}
        Statictis.append(ResultIHope)
    elif flag == False :
        ResultIHope = {str(m) : "Don't found suitable data"}
        Statictis.append(ResultIHope)
        

In [9]:
Statictis

[{'26/12/2022 - Monday': [{'Time to Call': '11h30',
    'Total Call': 9,
    'Total Reach In Day': 6,
    'Total Call In Time': 1,
    'Total Reach In Time': 1,
    'ConfidentInDays': 66.66666666666666,
    'ConfidentInTimes': 100.0},
   {'Time to Call': '11h15',
    'Total Call': 18,
    'Total Reach In Day': 12,
    'Total Call In Time': 1,
    'Total Reach In Time': 1,
    'ConfidentInDays': 66.66666666666666,
    'ConfidentInTimes': 100.0},
   {'Time to Call': '11h00',
    'Total Call': 27,
    'Total Reach In Day': 18,
    'Total Call In Time': 1,
    'Total Reach In Time': 1,
    'ConfidentInDays': 66.66666666666666,
    'ConfidentInTimes': 100.0},
   {'Time to Call': '10h30',
    'Total Call': 36,
    'Total Reach In Day': 24,
    'Total Call In Time': 0,
    'Total Reach In Time': 0,
    'ConfidentInDays': 66.66666666666666},
   {'Time to Call': '10h00',
    'Total Call': 45,
    'Total Reach In Day': 30,
    'Total Call In Time': 0,
    'Total Reach In Time': 0,
    'Confident